In [3]:
import os
import numpy as np
import librosa
import concurrent.futures
from tqdm import tqdm

In [4]:
def generate_and_save_mfcc(source_file, target_file):
    """
    Wczytuje plik MP3 (source_file),
    generuje MFCC,
    zapisuje do pliku .npy (target_file).
    """
    try:
        # Wczytanie pliku audio
        y, sr = librosa.load(source_file, sr=None)
        
        # Generowanie MFCC (np. 20 współczynników)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        
        # Zapis do pliku .npy
        np.save(target_file, mfcc)
    except Exception as e:
        print(f"[BŁĄD] Przetwarzanie pliku {source_file}: {e}")

def process_folders_mfcc(source_folder, target_folder, max_workers=4):
    """
    Rekurencyjnie przeszukuje katalog 'source_folder' w poszukiwaniu plików .mp3,
    generuje MFCC i zapisuje do .npy w analogicznej strukturze katalogów 'target_folder'.
    
    Wykorzystuje wielowątkowość (ThreadPoolExecutor) i pasek postępu (tqdm).
    """

    # Zgromadzimy listę plików (source -> target) do przetworzenia
    tasks = []
    
    for root, dirs, files in os.walk(source_folder):
        # Obliczamy relatywną ścieżkę względem katalogu źródłowego
        relative_path = os.path.relpath(root, source_folder)
        
        # Ścieżka docelowa (ten sam subfolder)
        target_dir = os.path.join(target_folder, relative_path)
        os.makedirs(target_dir, exist_ok=True)

        # Wybieramy pliki mp3 w danym folderze
        mp3_files = [f for f in files if f.lower().endswith('.mp3')]
        
        for filename in mp3_files:
            source_file = os.path.join(root, filename)
            
            # nazwa docelowego pliku (zamieniamy .mp3 -> .npy)
            base_name = os.path.splitext(filename)[0]
            target_file = os.path.join(target_dir, base_name + '.npy')
            
            tasks.append((source_file, target_file))
    
    # Teraz przetwarzamy z użyciem wątków i paska postępu
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # map pozwala nam zrównoleglić generate_and_save_mfcc
        # tqdm do paska postępu
        results = list(tqdm(
            executor.map(lambda x: generate_and_save_mfcc(x[0], x[1]), tasks),
            total=len(tasks),
            desc="Przetwarzanie plików (MFCC)",
            unit="plik"
        ))
    
    print(f"Zakończono przetwarzanie MFCC. Łącznie plików: {len(tasks)}")

In [5]:
source_path = "../../../datasets/jamendo/split_audio_dataset/"
target_path = "../../../datasets/jamendo/split_mfcc/"

process_folders_mfcc(source_path, target_path, max_workers=6)

Przetwarzanie plików (MFCC): 100%|██████████| 39097/39097 [06:37<00:00, 98.34plik/s] 

Zakończono przetwarzanie MFCC. Łącznie plików: 39097
